In [4]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import matplotlib
import time
import sklearn.model_selection
from sklearn.model_selection import train_test_split

In [26]:
train=pd.read_csv('train.csv')
train.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
y=train.review_scores_rating
x=train.drop(['review_scores_rating'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
# y: review_scores_rating

In [27]:
class ResidualBlock(nn.Module):
    def __init__(self, dims=432, layers=3, dropout=False, Lasso_regulize=False):
        super(ResidualBlock, self).__init__()
        conv_block=[]
        for layer in range(layers):
            conv_block += [ nn.Linear(dims, dims),
#                             nn.InstanceNorm1d(dims),
                            nn.ReLU(inplace=True),
                            ]

        self.conv_block = nn.Sequential(*conv_block)      
    def forward(self, x):
        return x + self.conv_block(x)

class network(nn.Module):
    def __init__(self, dim_in=432, dim_out=1, n_residual_blocks=9):
        super(network, self).__init__()    
        # Initial convolution block 

        model = [   nn.Linear(dim_in,dim_in ),
#                     nn.InstanceNorm1d(dim_in),
                    nn.ReLU(inplace=True) ]
#         # Downsampling
#         in_features = initial_features
#         out_features = initial_features*2
#         for _ in range(2):  
#             model += [  nn.Linear(in_features, out_features)
#                         nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
#                         nn.InstanceNorm2d(out_features),
#                         nn.ReLU(inplace=True) ]
#             in_features = out_features
#             out_features = in_features*2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(dims=dim_in)]

#         # Upsampling
#         out_features = in_features//2
#         for _ in range(2):
#             model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
#                         nn.InstanceNorm2d(out_features),
#                         nn.ReLU(inplace=True) ]
#             in_features = out_features
#             out_features = in_features//2

        # Output layer
        model += [ nn.Linear(dim_in,dim_out) 
                    ]
        self.model = nn.Sequential(*model)
    def forward(self, x):
        return self.model(x)
    

In [51]:
net=network()
optimizor=torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.5, 0.999))
loss_f=torch.nn.SmoothL1Loss()
epoch=300
start_time=time.time()
batch_size=256
batch_num=len(x_train)//batch_size
l1_lambda=0.5
for e in range(epoch):
    index=x_train.index.values
    np.random.shuffle(index)
    for i in range(batch_num):
        batch_index=index[i*batch_size: (i+1)*batch_size]
        x_batch=x_train.ix[batch_index].values #[batch,features]
        x_batch=torch.Tensor(x_batch)
        y_batch=y_train.ix[batch_index].values
        y_batch=torch.Tensor(y_batch)
        y_hat=net(x_batch).squeeze() 
#         print(y_batch.shape)
        l1_reg=None
        for W in net.parameters():
            if l1_reg is None:
                l1_reg = W.norm(1)
            else:
                l1_reg = l1_reg + W.norm(1)
        loss=loss_f(y_hat,y_batch)+l1_reg*l1_lambda
        optimizor.zero_grad()
        loss.backward()
        optimizor.step()
        print('epoch: ', e, 'batch: ', i, loss: ', loss)
    

tensor(1.00000e+05 *
       1.2611)
tensor(1.00000e+05 *
       1.2098)
tensor(1.00000e+05 *
       1.1593)
tensor(1.00000e+05 *
       1.1102)
tensor(1.00000e+05 *
       1.0617)
tensor(1.00000e+05 *
       1.0147)
tensor(96865.1875)
tensor(92371.5469)
tensor(87963.8750)
tensor(83684.7422)
tensor(79492.4141)
tensor(75437.6016)
tensor(71471.9766)
tensor(67621.2188)
tensor(63872.4062)
tensor(60260.8438)
tensor(56712.7578)
tensor(53305.5664)
tensor(49996.6523)
tensor(46809.5312)
tensor(43713.9297)
tensor(40739.6172)
tensor(37863.1367)
tensor(35112.6445)
tensor(32458.2637)
tensor(29913.8340)
tensor(27478.7129)
tensor(25169.5918)
tensor(22936.3086)
tensor(20838.2949)
tensor(18836.5996)
tensor(16962.0371)
tensor(15169.1670)
tensor(13505.5986)
tensor(11934.4980)
tensor(10493.0762)
tensor(9140.7275)
tensor(7908.5347)
tensor(6786.9380)
tensor(5770.4619)
tensor(4847.1968)
tensor(4061.1470)
tensor(3359.5317)
tensor(2785.0361)
tensor(2301.3047)
tensor(1946.0935)
tensor(1675.4286)
tensor(1550.2834

tensor(1428.5209)
tensor(1421.6259)
tensor(1421.5198)
tensor(1433.8630)
tensor(1425.4408)
tensor(1430.9648)
tensor(1416.6552)
tensor(1427.4934)
tensor(1430.4331)
tensor(1428.1044)
tensor(1415.0892)
tensor(1434.9849)
tensor(1421.4423)
tensor(1417.9780)
tensor(1419.2806)
tensor(1431.6454)
tensor(1429.8459)
tensor(1432.4003)
tensor(1415.0623)
tensor(1429.4775)
tensor(1427.7468)
tensor(1423.3440)
tensor(1416.8542)
tensor(1443.4995)
tensor(1425.6047)
tensor(1423.5701)
tensor(1417.4409)
tensor(1431.9984)
tensor(1434.5583)
tensor(1430.0192)
tensor(1413.0569)
tensor(1432.8640)
tensor(1428.5454)
tensor(1426.9882)
tensor(1417.7810)
tensor(1438.6530)
tensor(1423.0686)
tensor(1418.4133)
tensor(1424.0018)
tensor(1436.4489)
tensor(1430.4553)
tensor(1430.3773)
tensor(1414.7208)
tensor(1430.6780)
tensor(1429.7821)
tensor(1428.8378)
tensor(1419.5303)
tensor(1438.3037)
tensor(1421.1047)
tensor(1417.6101)
tensor(1424.4832)
tensor(1431.1027)
tensor(1427.5400)
tensor(1427.5978)
tensor(1428.0770)
tensor(142

tensor(1423.1810)
tensor(1414.0159)
tensor(1415.6210)
tensor(1431.9564)
tensor(1420.9670)
tensor(1429.5476)
tensor(1410.1543)
tensor(1432.7893)
tensor(1426.3204)
tensor(1424.8657)
tensor(1424.2543)
tensor(1440.2505)
tensor(1423.1390)
tensor(1416.4750)
tensor(1422.7222)
tensor(1429.1838)
tensor(1428.9327)
tensor(1428.8140)
tensor(1410.7736)
tensor(1421.7805)
tensor(1432.4344)
tensor(1419.9407)
tensor(1411.1708)
tensor(1440.9742)
tensor(1424.3916)
tensor(1415.5179)
tensor(1420.6130)
tensor(1428.4805)
tensor(1424.7074)
tensor(1429.4130)
tensor(1415.2996)
tensor(1426.8254)
tensor(1429.9921)
tensor(1420.0339)
tensor(1424.5645)
tensor(1438.6947)
tensor(1414.1289)
tensor(1417.6523)
tensor(1431.8528)
tensor(1427.7716)
tensor(1417.5278)
tensor(1432.4014)
tensor(1413.8934)
tensor(1423.9188)
tensor(1426.6099)
tensor(1421.1678)
tensor(1415.2858)
tensor(1439.3606)
tensor(1424.6631)
tensor(1416.3228)
tensor(1420.9788)
tensor(1439.1401)
tensor(1416.0092)
tensor(1429.6711)
tensor(1414.3857)
tensor(142

tensor(1420.3571)
tensor(1413.5027)
tensor(1420.3555)
tensor(1427.1879)
tensor(1419.6573)
tensor(1430.2809)
tensor(1410.6495)
tensor(1426.0248)
tensor(1426.2729)
tensor(1423.1550)
tensor(1421.1971)
tensor(1447.4182)
tensor(1417.3829)
tensor(1414.2996)
tensor(1418.7162)
tensor(1430.9954)
tensor(1424.9244)
tensor(1428.1078)
tensor(1414.6196)
tensor(1428.2736)
tensor(1424.7510)
tensor(1421.1167)
tensor(1423.4331)
tensor(1433.2742)
tensor(1418.5679)
tensor(1417.1783)
tensor(1416.7571)
tensor(1430.6403)
tensor(1420.8142)
tensor(1423.5938)
tensor(1412.6302)
tensor(1426.6608)
tensor(1424.2384)
tensor(1419.9058)
tensor(1415.0967)
tensor(1433.1212)
tensor(1418.3876)
tensor(1416.2660)
tensor(1419.7379)
tensor(1424.6805)
tensor(1420.3450)
tensor(1428.4961)
tensor(1413.1102)
tensor(1427.1685)
tensor(1428.4805)
tensor(1422.5870)
tensor(1413.8136)
tensor(1440.2095)
tensor(1416.5886)
tensor(1415.6562)
tensor(1417.1838)
tensor(1431.7751)
tensor(1423.0460)
tensor(1426.1243)
tensor(1418.9890)
tensor(143

tensor(1423.1609)
tensor(1412.4684)
tensor(1428.5287)
tensor(1428.1298)
tensor(1420.4758)
tensor(1424.5311)
tensor(1408.0643)
tensor(1421.5907)
tensor(1426.0406)
tensor(1418.0138)
tensor(1431.0243)
tensor(1434.1157)
tensor(1418.8362)
tensor(1416.3210)
tensor(1416.5156)
tensor(1430.1533)
tensor(1424.7871)
tensor(1430.9144)
tensor(1413.1116)
tensor(1426.2139)
tensor(1428.1166)
tensor(1418.9233)
tensor(1413.6809)
tensor(1433.4026)
tensor(1416.2839)
tensor(1419.5132)
tensor(1420.8835)
tensor(1427.1246)
tensor(1417.4680)
tensor(1429.6815)
tensor(1409.8848)
tensor(1424.9077)
tensor(1423.7762)
tensor(1420.3978)
tensor(1408.9854)
tensor(1448.6422)
tensor(1421.0359)
tensor(1410.0402)
tensor(1416.7522)
tensor(1431.2701)
tensor(1417.2173)
tensor(1427.2133)
tensor(1414.2063)
tensor(1426.6414)
tensor(1426.8751)
tensor(1423.3146)
tensor(1423.7665)
tensor(1440.0106)
tensor(1420.2302)
tensor(1412.5376)
tensor(1417.8826)
tensor(1432.0598)
tensor(1425.0992)
tensor(1426.8240)
tensor(1415.3202)
tensor(142

...

tensor(1429.0676)
tensor(1420.2845)
tensor(1430.5573)
tensor(1411.9877)
tensor(1426.0188)
tensor(1425.9922)
tensor(1419.9094)
tensor(1417.8934)
tensor(1436.5696)
tensor(1416.9091)
tensor(1413.6974)
tensor(1421.9491)
tensor(1423.5262)
tensor(1421.5055)
tensor(1428.0125)
tensor(1411.8011)
tensor(1426.0594)
tensor(1422.9685)
tensor(1416.7876)
tensor(1412.0896)
tensor(1430.8434)
tensor(1420.0258)
tensor(1418.3276)
tensor(1415.6545)
tensor(1427.5764)
tensor(1421.2261)
tensor(1421.0438)
tensor(1411.9708)
tensor(1422.6853)
tensor(1428.4116)
tensor(1418.6287)
tensor(1411.4587)
tensor(1433.3568)
tensor(1426.9095)
tensor(1432.7385)
tensor(1415.5421)
tensor(1427.1141)
tensor(1421.2590)
tensor(1422.5120)
tensor(1422.6664)
tensor(1420.9484)
tensor(1427.1165)
tensor(1420.5116)
tensor(1416.3258)
tensor(1431.9941)
tensor(1418.8319)
tensor(1414.8745)
tensor(1412.4281)
tensor(1427.8457)
tensor(1418.5200)
tensor(1424.1016)
tensor(1411.3168)
tensor(1422.2092)
tensor(1425.0205)
tensor(1422.0800)
tensor(141

tensor(1417.7371)
tensor(1430.6296)
tensor(1411.9778)
tensor(1437.4751)
tensor(1423.8953)
tensor(1420.2371)
tensor(1410.7317)
tensor(1434.2062)
tensor(1422.9865)
tensor(1418.2023)
tensor(1411.7688)
tensor(1429.7544)
tensor(1422.9718)
tensor(1423.5787)
tensor(1409.0850)
tensor(1425.6367)
tensor(1423.6328)
tensor(1418.0787)
tensor(1416.2264)
tensor(1432.0013)
tensor(1429.6036)
tensor(1413.4808)
tensor(1419.1754)
tensor(1427.6343)
tensor(1420.1841)
tensor(1423.8711)
tensor(1409.1770)
tensor(1420.6198)
tensor(1427.0609)
tensor(1416.1337)
tensor(1412.4232)
tensor(1431.4080)
tensor(1416.9478)
tensor(1416.4910)
tensor(1415.5780)
tensor(1432.4230)
tensor(1426.6370)
tensor(1438.0168)
tensor(1409.5358)
tensor(1420.4435)
tensor(1424.2231)
tensor(1420.2083)
tensor(1415.0540)
tensor(1444.7528)
tensor(1416.6283)
tensor(1417.6702)
tensor(1417.4224)
tensor(1426.7166)
tensor(1420.9589)
tensor(1426.3331)
tensor(1411.3395)
tensor(1425.5945)
tensor(1425.8687)
tensor(1418.2987)
tensor(1416.2747)
tensor(143

tensor(1422.2438)
tensor(1424.2793)
tensor(1407.0216)
tensor(1427.6125)
tensor(1424.9272)
tensor(1418.9015)
tensor(1411.0098)
tensor(1435.8574)
tensor(1415.5977)
tensor(1417.0900)
tensor(1420.4420)
tensor(1429.3563)
tensor(1423.4454)
tensor(1423.7153)
tensor(1407.6285)
tensor(1420.9187)
tensor(1424.5129)
tensor(1420.5284)
tensor(1413.1805)
tensor(1438.0804)
tensor(1419.0006)
tensor(1416.4233)
tensor(1419.2618)
tensor(1429.7562)
tensor(1422.3654)
tensor(1426.7180)
tensor(1408.7975)
tensor(1434.6331)
tensor(1428.6498)
tensor(1416.4657)
tensor(1410.4513)
tensor(1430.2462)
tensor(1422.5386)
tensor(1415.5298)
tensor(1419.4271)
tensor(1422.8528)
tensor(1416.9459)
tensor(1422.9865)
tensor(1413.4536)
tensor(1419.8370)
tensor(1421.5090)
tensor(1419.0944)
tensor(1414.5332)
tensor(1432.2145)
tensor(1417.4065)
tensor(1424.1626)
tensor(1419.2692)
tensor(1427.2998)
tensor(1420.6221)
tensor(1428.9244)
tensor(1408.9001)
tensor(1423.4946)
tensor(1441.2893)
tensor(1417.9260)
tensor(1409.6448)
tensor(143

tensor(1427.6282)
tensor(1408.8033)
tensor(1422.9266)
tensor(1424.3201)
tensor(1430.4375)
tensor(1412.4641)
tensor(1433.8424)
tensor(1419.7612)
tensor(1413.9166)
tensor(1415.4331)
tensor(1426.7719)
tensor(1418.1632)
tensor(1439.9966)
tensor(1411.1218)
tensor(1423.1327)
tensor(1431.6168)
tensor(1419.8218)
tensor(1412.5775)
tensor(1431.7783)
tensor(1412.9882)
tensor(1425.7438)
tensor(1427.4017)
tensor(1429.1803)
tensor(1419.9038)
tensor(1425.5515)
tensor(1409.6707)
tensor(1424.3790)
tensor(1429.5734)
tensor(1423.1195)
tensor(1411.4884)
tensor(1434.4872)
tensor(1421.4602)
tensor(1413.6292)
tensor(1415.0898)
tensor(1431.4113)
tensor(1422.0822)
tensor(1428.4127)
tensor(1407.4104)
tensor(1422.2410)
tensor(1424.2363)
tensor(1419.7480)
tensor(1413.1141)
tensor(1445.4192)
tensor(1417.2338)
tensor(1415.2046)
tensor(1416.8855)
tensor(1426.9374)
tensor(1417.3666)
tensor(1426.6252)
tensor(1410.1602)
tensor(1419.2114)
tensor(1424.4398)
tensor(1419.8292)
tensor(1410.2692)
tensor(1444.3490)
tensor(142

KeyboardInterrupt: 

In [57]:
# y_predict=net(torch.Tensor(x_test.values))
np.average(y_predict.detach().numpy()-y_test.values)

3753.8582975383456

In [48]:
net=network()
optimizor=torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.5, 0.999))
loss_f=torch.nn.SmoothL1Loss()
epoch=300
start_time=time.time()
batch_size=256
batch_num=len(x_train)//batch_size
l1_lambda=0.5
for e in range(epoch):
    index=x_train.index.values
    np.random.shuffle(index)
    for i in range(batch_num):
        batch_index=index[i*batch_size: (i+1)*batch_size]
        x_batch=x_train.ix[batch_index].values #[batch,features]
        x_batch=torch.Tensor(x_batch)
        y_batch=y_train.ix[batch_index].values
        y_batch=torch.Tensor(y_batch)
        y_hat=net(x_batch).squeeze() 
#         print(y_batch.shape)
        l1_reg=None
        for W in net.parameters():
            if l1_reg is None:
                l1_reg = W.norm(1)
            else:
                l1_reg = l1_reg + W.norm(1)
        loss=loss_f(y_hat,y_batch)+l1_reg*l1_lambda
        optimizor.zero_grad()
        loss.backward()
        optimizor.step()
        print(loss)

tensor(14.0107)
tensor(1.0199)
tensor(12.8451)
tensor(0.8873)
tensor(12.2491)
tensor(0.8489)
tensor(12.2830)
tensor(0.7769)
tensor(12.2728)
tensor(0.7280)
tensor(12.0601)
tensor(0.7408)
tensor(12.1048)
tensor(0.7288)
tensor(12.2337)
tensor(0.7190)
tensor(12.0645)
tensor(0.7340)
tensor(12.0913)
tensor(0.7936)
tensor(12.1897)
tensor(0.6866)
tensor(12.0695)
tensor(0.7589)
tensor(12.0963)
tensor(0.7494)
tensor(12.1623)
tensor(0.6835)
tensor(12.0471)
tensor(0.7376)
tensor(12.0335)
tensor(0.7583)
tensor(12.1493)
tensor(0.7119)
tensor(12.0674)
tensor(0.7365)
tensor(12.0454)
tensor(0.7537)
tensor(12.1435)
tensor(0.6969)
tensor(12.0542)
tensor(0.7616)
tensor(12.0671)
tensor(0.7616)
tensor(12.1463)
tensor(0.7307)
tensor(12.0208)
tensor(0.7538)
tensor(12.0704)
tensor(0.8778)
tensor(12.1358)
tensor(0.7155)
tensor(12.0464)
tensor(0.7289)
tensor(12.0336)
tensor(0.8435)
tensor(0.5941)
tensor(1.00000e-02 *
       4.9601)
